Create a autopilot car having LiDAR sensor attach on top.

Must run with generate_traffic.py file to spawn vehicles.

Collect cloud point data and store for futher investigation.

In [1]:
import glob
import os
import sys
import argparse
import time
from datetime import datetime
import random
import numpy as np
from matplotlib import cm
import open3d as o3d
import carla

from sklearn.cluster import DBSCAN
from scipy.spatial import KDTree
import cv2

from vehicle_features import vehicle_size_ranges, vehicle_centroid_colors, color_names, color_values

VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])


cloud_point_data = []
cloud_point = []

def generate_lidar_bp(blueprint_library, delta):
    """Generates a CARLA blueprint based on the script parameters"""
    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

    lidar_bp.set_attribute('upper_fov', str(10.0))
    lidar_bp.set_attribute('lower_fov', str(-30.0))
    lidar_bp.set_attribute('channels', str(64.0))
    lidar_bp.set_attribute('range', str(100.0))
    lidar_bp.set_attribute('rotation_frequency', str(20))
    lidar_bp.set_attribute('points_per_second', str(1000000))
    return lidar_bp

def lidar_callback(point_cloud, point_list):
    """Prepares a point cloud with intensity colors ready to be consumed by Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    # Isolate the intensity and compute a color for it
    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

    points = data[:, :-1]
    points[:, :1] = -points[:, :1]

    # Compute distances from the sensor
    distances = np.linalg.norm(points[:, :3], axis=1)

    # Apply filters for height (z > ground level) and distance (d > 30m)
    height_filter = points[:, 2] >= -1.7  # Filter points higher than the ground
    height_filter_1 = points[:, 2] <= 3  # Filter points higher than the ground
    # distance_filter = distances <= 15 

    # Combine the filters
    valid_points = np.logical_and(height_filter, height_filter_1)

    # Apply filters to points and colors
    filtered_points = points[valid_points]
    filtered_colors = int_color[valid_points]

    # Update the point cloud data
    point_list.points = o3d.utility.Vector3dVector(filtered_points)
    point_list.colors = o3d.utility.Vector3dVector(filtered_colors)

    cloud_point_data.clear()
    cloud_point_data.append(filtered_points)
    cloud_point.append(filtered_points)


class PointCloudProcessor:
    """
    Processes point cloud data frames, performs clustering, tracking, and vehicle classification.
    """
    def __init__(self, eps=0.5, min_samples=10):
        self.eps = eps
        self.min_samples = min_samples

        # Initialize tracking history and IDs
        self.previous_centroids = []
        self.previous_cluster_ids = []
        self.cluster_history = []
        self.tracking_id = 0
        self.cluster_color_map = {}
        self.previous_centroids_map = {}
        self.previous_directions_map = {}

        self.first = True

    def process_frames(self, cloud_point_frame):
        current_frame = cloud_point_frame
        # Perform clustering
        clustering = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit(current_frame)
        labels = clustering.labels_

        current_centroids = []
        cluster_points = []

        # Extract clusters and centroids
        for cluster_label in set(labels):
            if cluster_label == -1:
                continue  # Skip noise points
            clus_points = current_frame[labels == cluster_label]
            cluster_points.append(clus_points)

            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(clus_points)

            bbox = cluster_pcd.get_axis_aligned_bounding_box()
            bbox_center = bbox.get_center()
            current_centroids.append(bbox_center)

        current_centroids = np.array(current_centroids)

        # Track clusters
        if self.first == True:
            # For the first frame, initialize the cluster IDs
            cluster_ids = np.arange(len(current_centroids))
            self.tracking_id = len(current_centroids)
            self.first = False
        else:
            cluster_ids, self.tracking_id = self.track_clusters(current_centroids, self.previous_centroids, self.previous_cluster_ids)

        # Save the cluster history for visualization
        self.cluster_history.clear()
        self.cluster_history.append((cluster_points, current_centroids, cluster_ids))

        # Update previous centroids and cluster IDs
        self.previous_centroids = current_centroids
        self.previous_cluster_ids = cluster_ids

        # Assign colors to clusters
        for cluster_id in cluster_ids:
            self.assign_color(cluster_id)
        
    def track_clusters(self, current_centroids, previous_centroids, prev_cluster_ids, threshold=1.0):
        """Matches current centroids with previous centroids using nearest neighbor approach."""
        if len(previous_centroids) == 0:
            return np.arange(len(current_centroids)) + self.tracking_id, self.tracking_id + len(current_centroids)
        
        # Create a KDTree for fast nearest neighbor lookup
        tree = KDTree(previous_centroids)
        
        # Assign new tracking IDs based on nearest neighbors from the previous frame
        assigned_ids = np.full(len(current_centroids), -1)  # -1 for new clusters
        distances, indices = tree.query(current_centroids)
        
        # Assign tracking IDs based on distance threshold
        for i, (dist, idx) in enumerate(zip(distances, indices)):
            if dist < threshold:
                assigned_ids[i] = prev_cluster_ids[idx]  # Match with existing cluster ID
        
        # Assign new IDs to unmatched clusters
        new_clusters = (assigned_ids == -1)
        assigned_ids[new_clusters] = np.arange(self.tracking_id, self.tracking_id + np.sum(new_clusters))
        
        return assigned_ids, self.tracking_id + np.sum(new_clusters)
    
    def classify_vehicle(self, bbox):
        """Classifies the object as a car, truck, or motorcycle based on the bounding box dimensions."""
        dimensions = bbox.get_extent() 
        dimensions = sorted(dimensions, reverse=True)
        d0, d1, d2 = dimensions[0], dimensions[1], dimensions[2]
    
        for vehicle_type, size_range in vehicle_size_ranges.items():
            if (size_range['0'][0] <= d0 <= size_range['0'][1] and
                size_range['1'][0] <= d1 <= size_range['1'][1] and
                size_range['2'][0] <= d2 <= size_range['2'][1]):
                return vehicle_type
        return None

    def assign_color(self, cluster_id):
        """Assigns a color to a cluster if not already assigned."""
        if cluster_id not in self.cluster_color_map:
            color_index = len(self.cluster_color_map) % len(color_names)  # Cycle through the color list
            self.cluster_color_map[cluster_id] = (color_names[color_index], color_values[color_names[color_index]])
        
    def create_arrow(self, start, end, previous_direction=None, scale_factor=15.0):
        """
        Creates an arrow mesh representing movement direction with a scaled length based on the magnitude of the vector.
        Uses Exponential Moving Average (EMA) for smoothing the direction.
        """
        kmh = self.find_velocity_kmh(end, start)
        
        if kmh > 4:
            # Calculate the direction vector
            new_direction = end - start
            new_direction[2] = 0  # Ignore Z-axis movement for 2D plane movement visualization
    
            # If there is a previous direction, update it using exponential moving average
            if previous_direction is not None:
                updated_direction = 0.7 * previous_direction + 0.3 * new_direction  # Apply EMA
            else:
                updated_direction = new_direction
    
            # Calculate the magnitude (length) of the direction vector
            length = np.linalg.norm(updated_direction)
    
            if length == 0:
                return None  # No arrow needed if there's no movement
    
            # Scale the arrow length by a given factor
            scaled_length = length * scale_factor
    
            # Normalize the updated direction vector
            updated_direction_normalized = updated_direction / length
    
            # Create an arrow mesh with length proportional to the scaled movement
            arrow = o3d.geometry.TriangleMesh.create_arrow(cylinder_radius=0.1, cone_radius=0.2,
                                                           cylinder_height=scaled_length * 0.3, cone_height=scaled_length * 0.2)
    
            # Align the arrow with the updated direction vector
            z_unit_vector = np.array([0, 0, 1])  # Upward vector
            axis = np.cross(z_unit_vector, updated_direction_normalized)
            axis_length = np.linalg.norm(axis)
    
            if axis_length != 0:
                axis /= axis_length  # Normalize the axis of rotation
                angle = np.arccos(np.clip(np.dot(z_unit_vector, updated_direction_normalized), -1.0, 1.0))
                R = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle)
                arrow.rotate(R, center=(0, 0, 0))
    
            # Translate the arrow to start from the previous centroid (start position)
            arrow.translate(start)
            arrow.paint_uniform_color([1, 0, 1])  # Set arrow color to purple
    
            return arrow
        
        return None

    def find_velocity_kmh(self, start, end, time=1/20):
        """Computes velocity in km/h between two points."""
        v = np.linalg.norm(end - start) / time
        kmh = int(3.6 * v)
        return kmh


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19520\1341349337.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  VIRIDIS = np.array(cm.get_cmap('plasma').colors)


In [ ]:

client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
world = client.get_world()

original_settings = world.get_settings()
settings = world.get_settings()
traffic_manager = client.get_trafficmanager(8000)
traffic_manager.set_synchronous_mode(True)

delta = 0.05

settings.fixed_delta_seconds = delta
settings.synchronous_mode = True
settings.no_rendering_mode = False
world.apply_settings(settings)

blueprint_library = world.get_blueprint_library()
vehicle_bp = blueprint_library.filter("*model3*")[0]
spawn_points = world.get_map().get_spawn_points()

vehicle_transform = random.choice(world.get_map().get_spawn_points())

start_point = spawn_points[0]
vehicle = world.spawn_actor(vehicle_bp, start_point)

vehicle.set_autopilot(True)

lidar_bp = generate_lidar_bp(blueprint_library, delta)

lidar_transform = carla.Transform(carla.Location(x=0, z=2))

lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=vehicle)

point_list = o3d.geometry.PointCloud()


lidar.listen(lambda data: lidar_callback(data, point_list))

eps = 0.5
min_samples = 10
frame_rate = 20

processor = PointCloudProcessor(eps=eps, min_samples=min_samples)

vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True

# Access the ViewControl object
view_ctl = vis.get_view_control()


frame = 0
dt0 = datetime.now()
while True:
    if frame == 2:
        vis.add_geometry(point_list)
    
    if(cloud_point_data):
        processor.process_frames(cloud_point_data[0])
        cluster_points, centroids, cluster_ids = processor.cluster_history[0]

        all_points = []
        all_colors = []
        bounding_boxes = []
        centroids_spheres = []
        arrows = []  # To store arrows between previous and current centroids
        previous_centroids_spheres = []  # To store previous centroids as spheres

        for i, cluster in enumerate(cluster_points):
            # Add all points to the list for visualization
            all_points.append(cluster)
            
            # Create an Open3D point cloud for the current cluster
            cluster_pcd = o3d.geometry.PointCloud()
            cluster_pcd.points = o3d.utility.Vector3dVector(cluster)
            
            # Compute the bounding box for the cluster
            bbox = cluster_pcd.get_axis_aligned_bounding_box()
            bbox_center = bbox.get_center()

            # Ensure that the color is assigned
            processor.assign_color(cluster_ids[i])

            # Use bounding box center as centroid and classify for vehicle types
            vehicle_type = processor.classify_vehicle(bbox)
            
            if vehicle_type:
                # Add the bounding box to the list for visualization
                bbox.color = (0, 1, 0)  # Set bounding box color (e.g., green)
                bounding_boxes.append(bbox)

                centroid_color = vehicle_centroid_colors[vehicle_type]  # Color based on vehicle type

                # Set the color of the entire point cloud cluster to match the centroid color
                all_colors.append(np.tile(centroid_color, (len(cluster), 1)))  # Apply centroid color to the points

                # Check if we have a previous centroid for this cluster to draw an arrow
                if cluster_ids[i] in processor.previous_centroids_map:
                    prev_centroid = processor.previous_centroids_map[cluster_ids[i]]
                    
                    # Get the previous direction if it exists
                    previous_direction = processor.previous_directions_map.get(cluster_ids[i], None)

                    # Create and append the arrow with EMA
                    arrow = processor.create_arrow(prev_centroid, bbox_center, previous_direction)
                    if arrow:
                        arrows.append(arrow)
                    
                    # Store the new direction for future EMA updates
                    new_direction = bbox_center - prev_centroid
                    processor.previous_directions_map[cluster_ids[i]] = new_direction  # Update the direction map

                    # Create a sphere at the previous centroid (grey color)
                    prev_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
                    prev_sphere.translate(prev_centroid)
                    prev_sphere.paint_uniform_color([0.5, 0.5, 0.5])  # Grey color for previous centroids
                    previous_centroids_spheres.append(prev_sphere)
                
                # Update the previous centroid for this cluster
                processor.previous_centroids_map[cluster_ids[i]] = bbox_center
                
            else:
                centroid_color = [0, 0, 1]  # Default blue if no vehicle classification
                all_colors.append(np.tile(centroid_color, (len(cluster), 1)))  # Apply default color to points
            
            # Create a sphere at the centroid with the appropriate vehicle type color
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)
            sphere.translate(bbox_center)
            sphere.paint_uniform_color(centroid_color)  # Set centroid color based on vehicle type
            centroids_spheres.append(sphere)

        if len(all_points) > 0:
            # Flatten the points and colors arrays for visualization
            all_points = np.vstack(all_points)
            all_colors = np.vstack(all_colors)
            
            # Create an Open3D point cloud object for visualization
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(all_points)
            pcd.colors = o3d.utility.Vector3dVector(all_colors)

        vis.clear_geometries()
        entities_to_draw = [pcd] + bounding_boxes + arrows

        for entity in entities_to_draw:
            vis.add_geometry(entity)


        view_ctl.set_zoom(0.1)
        vis.poll_events()
        vis.update_renderer()
        # vis.run()
    
    # Check if the window is closed by the user
    if not vis.poll_events():  # If the window is closed
        print("Window closed by user. Terminating...")
        world.apply_settings(original_settings)
        traffic_manager.set_synchronous_mode(False)

        vehicle.destroy()
        lidar.destroy()
        vis.destroy_window()
        break

    # Fix Open3D jittering issues
    time.sleep(0.005)
    world.tick()

    process_time = datetime.now() - dt0
    sys.stdout.write('\r' + 'FPS: ' + str(1.0 / process_time.total_seconds()))
    sys.stdout.flush()
    dt0 = datetime.now()
    frame += 1

# Destroy the visualizer window
vis.destroy_window()

In [9]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

world.apply_settings(original_settings)
traffic_manager.set_synchronous_mode(False)
lidar.destroy()

False

In [3]:
import pickle

# Save cloud_point_data as a pickle file
output_file = 'E:\\SelfDrive-master\\21-10-2024\\data\\cloud_point_data.pkl'

with open(output_file, 'wb') as f:
    pickle.dump(cloud_point, f)

print(f"Point cloud data saved to {output_file}")
print(f"Number of point clouds saved: {len(cloud_point)}")


import pickle

# Load the pickle file
with open('E:\\SelfDrive-master\\21-10-2024\\data\\cloud_point_data.pkl', 'rb') as f:
    cloud_point_test = pickle.load(f)

print(f"Loaded {len(cloud_point_test)} point clouds")

print(len(cloud_point_test))
print(cloud_point_test[0])


Point cloud data saved to E:\SelfDrive-master\21-10-2024\data\cloud_point_data.pkl
Number of point clouds saved: 2652
